## Exercise 11: XPS with GW

During the lecture you have learned a graphical way to get the HOMO state quasi-particle energy of benzene from the G<sub>0</sub>W<sub>0</sub> calculation. The solutions are the intersections between a special strainght line and the real part of the correlation part of the self energy Σ<sup>c</sup>. It looks like Figure 11. A in [this review](https://www.frontiersin.org/articles/10.3389/fchem.2019.00377/full). The x-coordinates of the intersection points corresponding to the quasi-particle energies. 

In principle, the best result should be only one intersection point with relative flat slope of Σ<sup>c</sup> at that point. A sharp slope at the intersection point corresponds to a week peak in the XPS, whereas a flat slope corresponds to a sharp peak in the XPS. Two or more intersection points with flat slopes might be a sign of G<sub>0</sub>W<sub>0</sub> is insufficient for this calculation. 

Anyhow, there is also a self-consistent way to get the quasi-particle energy too. More specifically, we will use analytic continuation techinque to obtain an analytical expression of the Σ<sup>c</sup> and keep iterating the quasi-particle energy using this analytical expression to get the final converged Σ<sup>c</sup> and quasi-particle energy. 

In this exercise, you will need to complete the iteration code to get the converged HOMO state quasi-particle energy of the benzene, and compare this result with the graphical solution you get from the lecture.

In [ ]:
# install numpy and matplotlib first if you run this code on binder
!pip install numpy
!pip install matplotlib

In [9]:
import numpy as np
import matplotlib.pyplot as plt

You could use the following function `get_real_self_energy` as a black box. It is a pre-calculated analytical expression of the Σ<sup>c</sup> of benzene. The input of this function is the quasi-particle enery minus the chemical potential energy, and the output is the Σ<sup>c</sup>. 

The result is converged meaning that the difference between the input quasi-particle energy and the quasi-particle energy calculated from the output Σ<sup>c</sup> are smaller than the given threshold. 

In [10]:
# This is a pre-calculated analytical expression of the Σ^c of the benzene.
# The input of this function is the quasi-particle energy - chemical potential energy, 
# and it returns the Σ<sup>c</sup>. I/O unit is in eV.
def get_real_self_energy(qp_energy):
    n_freq = 120
    n_par = 16
    n_step = n_freq/(n_par-1)
    i_dat = int(0)
    har_to_ev = 27.2114
    qp_energy = qp_energy/har_to_ev
    
    omega = np.array([  4.9788542564589873E-005  ,   
 2.6240722435716163E-004  ,
 6.4522816234115178E-004  ,
 1.1988574930602190E-003  ,
 1.9240604810920215E-003  ,
 2.8218303112733699E-003  ,
 3.8933972318023380E-003  ,
 5.1402333088764324E-003  ,
 6.5640570486457861E-003  ,
 8.1668385580586384E-003  ,
 9.9508054142374968E-003  ,
 1.1918449312612458E-002  ,
 1.4072533541396032E-002  ,
 1.6416101326234514E-002  ,
 1.8952485090843484E-002  ,
 2.1685316683558477E-002  ,
 2.4618538624922214E-002  ,
 2.7756416437340368E-002  ,
 3.1103552124375239E-002  ,
 3.4664898874412230E-002  ,
 3.8445777071269005E-002  ,
 4.2451891702888371E-002  ,
 4.6689351268646130E-002  ,
 5.1164688296106375E-002  ,
 5.5884881589379945E-002  ,
 6.0857380343709203E-002  ,
 6.6090130274652034E-002  ,
 7.1591601925425949E-002  ,
 7.7370821332780504E-002  ,
 8.3437403250386566E-002  ,
 8.9801587149397996E-002  ,
 9.6474276238809151E-002  ,
0.10346707977379387       ,
0.11079235894870215       ,
0.11846327670319039       ,
0.12649385180550279       ,
0.13489901761670264       ,
0.14369468598424232       ,
0.15289781676331174       ,
0.16252649352066642       ,
0.17260000603897507       ,
0.18313894031113342       ,
0.19416527679462228       ,
0.20570249778716992       ,
0.21777570488826112       ,
0.23041174762820069       ,
0.24363936447959167       ,
0.25748933761763843       ,
0.27199466296849673       ,
0.28719073728228017       ,
0.30311556419320307       ,
0.31980998148829087       ,
0.33731791210352186       ,
0.35568664170857001       ,
0.37496712613606659       ,
0.39521433236746051       ,
0.41648761731580736       ,
0.43885114925890617       ,
0.46237437748934618       ,
0.48713255657949750       ,
0.51320733263123897       ,
0.54068740001874427       ,
0.56966923846998774       ,
0.60025794190763215       ,
0.63256815232995189       ,
0.66672511421516190       ,
0.70286586754876279       ,
0.74114060069029397       ,
0.78171418802059245       ,
0.82476794177633284       ,
0.87050161285067718       ,
0.91913568182385907       ,
0.97091398934444517       ,
 1.0261067645369808       ,
 1.0850141217773648       ,
 1.1479701104779934       ,
 1.2153474201303209       ,
 1.2875628646229920       ,
 1.3650837968991019       ,
 1.4484356387771338       ,
 1.5382107531176799       ,
 1.6350789389426270       ,
 1.7397998978710683       ,
 1.8532381066728099       ,
 1.9763806416805181       ,
 2.1103586441085427       ,
 2.2564733016990117       ,
 2.4162274662295049       ,
 2.5913643485768008       ,
 2.7839151615893898       ,
 2.9962581559468866       ,
 3.2311922724036006       ,
 3.4920296972878861       ,
 3.7827130762349865       ,
 4.1079651898924103       ,
 4.4734817877382538       ,
 4.8861824106729665       ,
 5.3545400226600615       ,
 5.8890190748081626       ,
 6.5026647669667748       ,
 7.2119062255374589       ,
 8.0376671770578874       ,
 9.0069264007610883       ,
 10.154948829778068       ,
 11.528538118585407       ,
 13.190882293361229       ,
 15.228950835024138       ,
 17.765102443323034       ,
 20.975883140723901       ,
 25.123594482342398       ,
 30.611600587269130       ,
 38.086201589545425       ,
 48.635924670634409       ,
 64.211274913828802       ,
 88.594980003310624       ,
 129.93354546636172       ,
 208.53187426125751       ,
 387.45983915658258       ,
 952.71767235998732       ,
 5021.2355518476770     ])
    
    par = np.array([ complex(  -4.0887219661880178E-002  ,  -8.8331860693554783E-006 ), 
complex(  -4.3352824931516807       ,  0.12062101645845649      ),
complex(   4.2351179231386755       , -0.19533861494581156      ),
complex(  0.57478577141552056       ,   4.9239330646972668E-002 ),
complex(  -1.0969065433666181       , -0.52314987787371259      ),
complex(  -1.5593294786323835       ,  0.62403113064888782      ),
complex(   1.0372352247989687       ,  -1.4435185677691975      ),
complex(  0.53801539882142690       ,  0.32481653984405306      ),
complex(   1.1202427694042496E-002  , -0.71159802617610823      ),
complex( -0.23216517587459004       , -0.22418932583380621      ),
complex( -0.11561420736847916       ,   6.1350053461909262E-002 ),
complex(  0.13727367182020164       , -0.87919273536255149      ),
complex(  -5.6287280518628122E-003  ,  0.14178834208802901      ),
complex(   1.5776468677250999E-003  ,  -5.0629189188886939E-002 ),
complex(   5.2472938129254111E-003  ,   4.8845191308505113E-003 ),
complex(  -1.5871080176235082E-003  ,  -1.8220916175371853E-002 )])
    
    xdata=np.array([])
    for i_par in np.arange(n_par-1):
        xdata = np.append(xdata,complex(0.0,omega[i_dat]))
        i_dat = int(i_dat + n_step)
    
    xdata = np.append(xdata,complex(0.0,omega[n_freq-1]))

    gtmp = complex(1.0,0.0)
    for i_par in np.arange(n_par, 1, -1):
        gtmp = 1.0 + par[i_par-1]*(complex(qp_energy,0.0)-xdata[i_par-2])/gtmp
        
    self_energy = par[0]*har_to_ev/gtmp
    
    return self_energy
    

In [11]:
# setup the constants

# hatree to eV
har_to_ev = 27.2114

# Convergence threshold
qp_energy_thr = 10**(-5) * har_to_ev 

# Chemical potential of the benzene system, this value needs to be subtracted from the quasi-particle energy 
# before putting into the get_real_self_energy function.
# This is a pre-calculated value too. The unit is eV.
chemical_potential = -3.8526

# The exchange correlation potential v_xc. The unit is eV.
v_xc = -13.5051

# The exchange part of the self-energy sigma_x. The unit is eV.
sigma_x = -16.0698

# The Kohn-Sham energy e_ks. The unit is eV.
e_ks = -6.3048

The quasi-particle energy is calculated based on: 

**qp_energy = e_ks + sigma_x - v_xc + real(Σ^c**)


Please finish the remaining part of the code. 

***Hints***: The iteration loop should be like this;
1. Using *e_ks* as initial qp_energy, subtract chemical potential, i.e. *qp_energy* = *qp_energy* - *chemical_potential*, call `get_real_self_energy(qp_energy)`, and obtain the first exchange self energy Σ^c.
2. Put previous Σ^c into above *qp_energy* expression and get next *qp_energy*, subtract chemical_potential and then call `get_real_self_energy(qp_energy)` again. 
3. Keep running this loop until the qp_energy is converged, i.e. the difference is smaller than *qp_energy_thr*.
4. This final qp_energy is the HOMO state quasi-particle energy of benzene.

Please compare this qp_energy with the one you get from the graphical solution, they should be the same or very close.